# Example usage of [stable-dreamfusion](https://github.com/ashawkey/stable-dreamfusion)
Pure-pytorch version

### Check the machine

### Основные отличия от реализации из статьи
- Поскольку модель Imagen не является общедоступной, в данной реализации используется Stable Diffusion для ее замены (реализация от diffusers). В отличие от Imagen, Stable-Diffusion - это модель скрытой диффузии, которая диффундирует в скрытом пространстве вместо исходного пространства изображения. Такой выбор модели приводит к дополнительным временным затратам на обучение. В настоящее время 10000 тренировочных шагов занимают около 3 часов для тренировки на V100.
- Также здесь используется grid encoder с несколькими разрешениями для реализации NeRF backbone (реализация от torch-ngp), которая обеспечивает гораздо более быстрый рендеринг (~10 кадров в секунду при 800x800). Нормали поверхности предсказываются как в Magic3D.
- Vanilla NeRF backbone также поддерживается в настоящее время, но Mip-NeRF, представленная в статье, все еще не реализована.
- По умолчанию используется Adan optimizer.

In [ ]:
! nvidia-smi

Tue Feb 21 21:22:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    30W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Setup

In [ ]:
#@title install dependencies
! git clone https://github.com/ashawkey/stable-dreamfusion.git

%cd stable-dreamfusion

# install requirements
! pip install -r requirements.txt
! pip install git+https://github.com/NVlabs/nvdiffrast/

Cloning into 'stable-dreamfusion'...
remote: Enumerating objects: 496, done.
remote: Counting objects: 100% (256/256), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 496 (delta 193), reused 179 (delta 165), pack-reused 240
Receiving objects: 100% (496/496), 215.05 KiB | 19.55 MiB/s, done.
Resolving deltas: 100% (308/308), done.
/content/stable-dreamfusion
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.6/676.6 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.9/276.9 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Training & Testing
* It takes about 0.7s per training step, so the default 5000 training steps take around 1 hour to finish. A larger `Training_iters` usually leads to better results.
* If CUDA OOM, try to decrease `Num_steps`, `Upsample_steps`, and `Training_nerf_resolution`.
* If the NeRF fails to learn anything (empty scene, only background), try to decrease `Lambda_entropy` which regularizes the learned opacity.

In [ ]:
#@markdown ####**Training Settings:**
Prompt_text = "a green volcano" #@param {type: 'string'}
Training_iters = 3000 #@param {type: 'integer'}
Learning_rate = 1e-3 #@param {type: 'number'}
Training_nerf_resolution = 64  #@param {type: 'integer'}
# CUDA_ray = True #@param {type: 'boolean'}
# View_dependent_prompt = True #@param {type: 'boolean'}
# FP16 = True #@param {type: 'boolean'}
Seed = 0 #@param {type: 'integer'}
Lambda_entropy = 1e-4 #@param {type: 'number'}
Num_steps = 64 #@param {type: 'number'}
Upsample_steps = 32 #@param {type: 'number'}
Checkpoint = 'latest' #@param {type: 'string'}

#@markdown ---

#@markdown ####**Output Settings:**
Workspace = "trial" #@param{type: 'string'}
# Save_mesh = True #@param {type: 'boolean'}

# processings
Prompt_text = "'" + Prompt_text + "'"

In [ ]:
#@title start training
import torch
torch.cuda.empty_cache()

%run main.py -O2 --albedo --text {Prompt_text} --workspace {Workspace} --iters {Training_iters} --lr {Learning_rate} --w {Training_nerf_resolution} --h {Training_nerf_resolution} --seed {Seed} --lambda_entropy {Lambda_entropy} --ckpt {Checkpoint} --save_mesh --num_steps {Num_steps} --upsample_steps {Upsample_steps}

Namespace(H=800, O=False, O2=True, W=800, albedo=True, albedo_iters=3000, angle_front=60, angle_overhead=30, backbone='vanilla', bg_radius=1.4, blob_density=10, blob_radius=0.5, bound=1, ckpt='latest', cuda_ray=False, decimate_target=100000.0, density_activation='softplus', density_thresh=0.1, dir_text=True, dt_gamma=0, eval_interval=1, fovy=60, fovy_range=[40, 70], fp16=True, gui=False, guidance='stable-diffusion', h=64, hf_key=None, iters=3000, jitter_pose=False, lambda_entropy=0.0001, lambda_opacity=0, lambda_orient=0.01, lambda_tv=1e-07, light_phi=0, light_theta=60, lr=0.001, max_ray_batch=4096, max_spp=1, max_steps=1024, mcubes_resolution=256, min_lr=0.0001, min_near=0.1, negative='', num_steps=64, optim='adan', radius=3, radius_range=[1.0, 1.5], save_mesh=True, sd_version='2.1', seed=0, suppress_face=False, test=False, text='a green volcano', uniform_sphere_rate=0.5, update_extra_interval=16, upsample_steps=32, w=64, warm_iters=500, workspace='trial')
NeRFNetwork(
  (encoder): Fr

[INFO] Cmdline: main.py -O2 --albedo --text a green volcano --workspace trial --iters 3000 --lr 0.001 --w 64 --h 64
--seed 0 --lambda_entropy 0.0001 --ckpt latest --save_mesh --num_steps 64 --upsample_steps 32

[INFO] Trainer: df | 2023-02-21_23-17-58 | cuda | fp16 | trial

[INFO] #parameters: 25223

[INFO] Loading latest checkpoint ...

...

[INFO] load at epoch 30, global step 3000

[INFO] loaded optimizer.

[INFO] loaded scheduler.

[INFO] loaded scaler.

[INFO] training takes 0.0000 minutes.

In [ ]:
#@markdown ####**Testing Settings:**

Workspace_test = "trial" #@param{type: 'string'}
# Save_mesh = True #@param {type: 'boolean'}

In [ ]:
#@title testing 
import torch
torch.cuda.empty_cache()

%run main.py -O2 --test --workspace {Workspace_test} --save_mesh --max_ray_batch 1024

Namespace(H=800, O=False, O2=True, W=800, albedo=False, albedo_iters=1000, angle_front=60, angle_overhead=30, backbone='vanilla', bg_radius=1.4, blob_density=10, blob_radius=0.5, bound=1, ckpt='latest', cuda_ray=False, decimate_target=100000.0, density_activation='softplus', density_thresh=0.1, dir_text=True, dt_gamma=0, eval_interval=1, fovy=60, fovy_range=[40, 70], fp16=False, gui=False, guidance='stable-diffusion', h=64, hf_key=None, iters=10000, jitter_pose=False, lambda_entropy=0.0001, lambda_opacity=0, lambda_orient=0.01, lambda_tv=1e-07, light_phi=0, light_theta=60, lr=0.001, max_ray_batch=1024, max_spp=1, max_steps=1024, mcubes_resolution=256, min_lr=0.0001, min_near=0.1, negative='', num_steps=64, optim='adan', radius=3, radius_range=[1.0, 1.5], save_mesh=True, sd_version='2.1', seed=0, suppress_face=False, test=True, text=None, uniform_sphere_rate=0.5, update_extra_interval=16, upsample_steps=32, w=64, warm_iters=500, workspace='trial')
NeRFNetwork(
  (encoder): FreqEncoder_t

[INFO] Cmdline: main.py -O2 --test --workspace trial --save_mesh --max_ray_batch 1024

[INFO] Trainer: df | 2023-02-21_23-17-58 | cuda | fp32 | trial

[INFO] #parameters: 25223

[INFO] Loading latest checkpoint ...

...

==> Finished Test.

100% 100/100 [18:06<00:00, 10.86s/it]


==> Saving mesh to trial/mesh

[INFO] marching cubes thresh: 2.5 (1.1946775110802136e-21 ~ 37.626312255859375)
[INFO] mesh cleaning: (77030, 3) --> (75680, 3), (153494, 3) --> (150803, 3)
[INFO] mesh decimation: (75680, 3) --> (50181, 3), (150803, 3) --> (100000, 3)
[INFO] running xatlas to unwrap UVs for mesh: v=(50181, 3) f=(100000, 3)
[INFO] writing obj mesh to trial/mesh/mesh.obj
[INFO] writing vertices (50181, 3)
[INFO] writing vertices texture coords (62507, 2)
[INFO] writing faces (100000, 3)


==> Finished saving mesh.

### Display results
* RGB and Depth video are located at `{Workspace}/results/*.mp4`
* Mesh is under `{Workspace}/mesh/`, you could see three files named `mesh.obj`, `mesh.mtl`, and `albedo.png`.

In [ ]:
#@title display RGB video
import os
import glob
from IPython.display import HTML
from base64 import b64encode

def get_latest_file(path):
  dir_list = glob.glob(path)
  dir_list.sort(key=lambda x: os.path.getmtime(x))
  return dir_list[-1]

def show_video(video_path, video_width = 600):
   
  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"

  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
 
rgb_video = get_latest_file(os.path.join(Workspace, 'results', '*_rgb.mp4'))
show_video(rgb_video)

In [ ]:
#@title display RGB video
import os
import glob
from IPython.display import HTML
from base64 import b64encode

def get_latest_file(path):
  dir_list = glob.glob(path)
  dir_list.sort(key=lambda x: os.path.getmtime(x))
  return dir_list[-1]

def show_video(video_path, video_width = 600):
   
  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"

  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
 
rgb_video = get_latest_file(os.path.join(Workspace, 'results', '*_rgb.mp4'))
show_video(rgb_video)